* In this file we will create a matrix which shows the distance for each dealer from all the starting nodes in the 2013 q1 sold to network
* We also have to find these start nodes
    

* Step 0: Import Graphlab, sgraph etc
* Step 1: Import nodes list and edge list from csv files
* Step 2: Create SGraph from imported nodes list and edge list
* Step 3: Save the graph in a directory "sold_to_2013_q1". Then load the graph into a new Sgraph
* Step 4: Find starting nodes
    - We define these start nodes as the nodes which have indegree zero and outdegree greater than zero
    - Another issue is that we have to include dealers which have a self loop and are starting nodes. Graphlab, by design, counts their indegree as 1. As a result we will add them separately
* Step 5: Calculating distance from starting nodes
    - The files were becoming huge, so we had to split the data in sets of 100 nodes.
    - Will merge these files in stata


* Step 0: Import Graphlab, sgraph etc
* Step 1: Import nodes list and edge list from csv files

In [22]:
import graphlab 
from graphlab import SGraph, Vertex, Edge, SFrame, degree_counting

In [23]:
edge_data = SFrame.read_csv( 'H:/graphlab/2013_q1/SoldTo/edge_list.csv')
node_data=SFrame.read_csv('H:/graphlab/2013_q1/SoldTo/nodes.csv')

Finished parsing file H:\graphlab\2013_q1\SoldTo\edge_list.csv

Parsing completed. Parsed 100 lines in 1.17707 secs.

Finished parsing file H:\graphlab\2013_q1\SoldTo\edge_list.csv

Parsing completed. Parsed 1394933 lines in 0.78125 secs.

Finished parsing file H:\graphlab\2013_q1\SoldTo\nodes.csv

Parsing completed. Parsed 100 lines in 0.125002 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,long,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
------------------------------------------------------

Finished parsing file H:\graphlab\2013_q1\SoldTo\nodes.csv

Parsing completed. Parsed 250396 lines in 0.136818 secs.


Inferred types from first line of file as 
column_type_hints=[long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [3]:
node_data.head()

DealerTIN,indegree,outdegree,network_number,self_loop
427816,3,0,1,0
489461,2,2,1,0
427810,28,3,1,0
160199,9,0,1,0
427809,125,0,1,0
155446,8,7,1,0
252192,3,39,1,0
300114,1,2,1,0
427796,3,8,1,0
395109,9,16,1,0


* Step 2: Create SGraph from imported nodes list and edge list
* Step 3: Save the graph in a directory "sold_to_2013_q1". Then load the graph into a new Sgraph

In [24]:
g = SGraph(vertices=node_data, edges=edge_data, vid_field='DealerTIN', src_field='Source', dst_field='Destination')
g.save('H:/graphlab/2013_q1/SoldTo/Sold_to_2013_q1')
new_graph = graphlab.load_sgraph('H:/graphlab/2013_q1/SoldTo/Sold_to_2013_q1')

In [5]:
new_graph.summary()

{'num_edges': 1394933L, 'num_vertices': 250396L}

* Step 4: Find starting nodes
    - We define these start nodes as the nodes which have indegree zero and outdegree greater than zero


In [16]:
deg = degree_counting.create(new_graph) # count degrees of each node
deg_graph = deg['graph'] # a new SGraph with degree data attached to each vertex

sub_verts=deg_graph.get_vertices(fields={'in_degree': 0})
sub_verts=sub_verts[sub_verts['out_degree']>0]

In [18]:
print sub_verts.unique()
print len(sub_verts)

+--------+-----------+------------+--------------+
|  __id  | in_degree | out_degree | total_degree |
+--------+-----------+------------+--------------+
| 451737 |     0     |     2      |      2       |
| 397670 |     0     |     4      |      4       |
|  8143  |     0     |     12     |      12      |
|  3871  |     0     |     16     |      16      |
| 120032 |     0     |     2      |      2       |
| 398739 |     0     |     2      |      2       |
| 300699 |     0     |     2      |      2       |
| 166073 |     0     |     2      |      2       |
| 347654 |     0     |     2      |      2       |
| 148502 |     0     |     2      |      2       |
+--------+-----------+------------+--------------+
[14678 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
14678


- Another issue is that we have to include dealers which have a self loop and are starting nodes.
- Graphlab, by design, counts their indegree as 1. As a result we will add them separately
    - We created this list in stata, condition was indegree=0&outdegree>0&selfloop=1

In [19]:
self_loop_nodes=SFrame.read_csv('H:/graphlab/2013_q1/SoldTo/IndegreeZeroOutdegreeNonzeroWithSelfLoop.csv')
sub_verts=sub_verts.append(self_loop_nodes)

Finished parsing file H:\graphlab\2013_q1\SoldTo\IndegreeZeroOutdegreeNonzeroWithSelfLoop.csv

Parsing completed. Parsed 18 lines in 0.025997 secs.

Finished parsing file H:\graphlab\2013_q1\SoldTo\IndegreeZeroOutdegreeNonzeroWithSelfLoop.csv

Parsing completed. Parsed 18 lines in 0.024997 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [20]:
print sub_verts.unique()
print len(sub_verts)

+--------+-----------+------------+--------------+
|  __id  | in_degree | out_degree | total_degree |
+--------+-----------+------------+--------------+
| 451737 |     0     |     2      |      2       |
| 397670 |     0     |     4      |      4       |
|  8143  |     0     |     12     |      12      |
|  3871  |     0     |     16     |      16      |
| 120032 |     0     |     2      |      2       |
| 398739 |     0     |     2      |      2       |
| 300699 |     0     |     2      |      2       |
| 166073 |     0     |     2      |      2       |
| 347654 |     0     |     2      |      2       |
| 148502 |     0     |     2      |      2       |
+--------+-----------+------------+--------------+
[14696 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
14696


In [21]:
sub_verts_id=sub_verts.select_column('__id')
sub_verts.export_csv('H:/graphlab/2013_q1/SoldTo/StartNodes.csv')
sub_verts_id.save('H:/graphlab/2013_q1/SoldTo/StartNodesSarray.csv')

In [ ]:
parent_sframe=graphlab.SFrame()
i=1
for StartNode in sub_verts_id:
    if not parent_sframe.column_names():
        sp = graphlab.shortest_path.create(new_graph, source_vid=StartNode, verbose=False) # finds shortest path for all nodes
        sp_sframe = sp['distance'] 
        parent_sframe= sp_sframe
        parent_sframe.rename({'distance':'d'+str(StartNode)})
        i=i+1        
    else:
        sp = graphlab.shortest_path.create(new_graph, source_vid=StartNode, verbose=False)
        sp_sframe = sp['distance'] 
        sp_sframe.rename({'distance':'d'+str(StartNode)})
        parent_sframe = parent_sframe.join(sp_sframe,on='__id',how='outer')
        if i%100==0:
            parent_sframe.export_csv('H:/graphlab/2013_q1/SoldTo/StartNodes/DistanceMatrixFromStartNodes'+str(i)+'.csv')
            parent_sframe=graphlab.SFrame()
            i=i+1
        else:
            i=i+1
parent_sframe.export_csv('H:/graphlab/2013_q1/SoldTo/StartNodes/DistanceMatrixFromStartNodes'+str(i)+'.csv')    